In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

class esf:
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    es_index=IndicesClient(es)
    index = "platform"
    def __init__(self,doc_type):
        self.index = self.index
        self.doc_type = doc_type
    def must(self,must_cond,y):
        query = []
        for cond in must_cond:
            query.append({"match_phrase":{"_all":cond}},)
        query.append(y)
        return query

    def should(self,should_cond):
        query = []
        for cond in should_cond:
            query.append({"match_phrase":{"_all":cond}},)
        return query

    def must_not(self,mustnot_cond):
        query = []
        for cond in mustnot_cond:
            query.append({"match_phrase":{"_all":cond}},)
        return query
    
class esc(esf):
    es = esf.es
    es_index = esf.es_index
    
    def __init__(self,doc_type,must_cond,should_cond,mustnot_cond,time_sort,date1,date2):
        super().__init__(doc_type)
        self.index = esf.index
        self.time_sort = time_sort
        self.booling={"filtered": {
            "filter": {"bool":{
            "must":esf.must(self,must_cond,{"range":{"time":{"gte": date1 ,"lte": date2 }}}),
            "should":esf.should(self,should_cond),
            "must_not":esf.must_not(self,mustnot_cond),   
                    }}}}
        self.sort={"time": { "order": "desc" }}
        print (self.booling)

    def result(self,size):
        if self.time_sort == True:            
            conclution = self.es.search(index=self.index,doc_type=self.doc_type,
            body={"size":size,"query":self.booling,"sort":self.sort   
                  })
        else:
            conclution = self.es.search(index=self.index,doc_type=self.doc_type,
            body={"size":size,"query":self.booling})
        return conclution['hits']['hits']

    def count(self):
        conclution = self.es.count(index=self.index,doc_type=self.doc_type,
        body={"query":self.booling})
        return conclution["count"]

    def search(self):
        if self.time_sort == True:
            conclution = self.es.search(index=self.index,doc_type=self.doc_type,
            body={"size":10000000,"query":self.booling,"sort":self.sort   
                  })
        else:
            conclution = self.es.search(index=self.index,doc_type=self.doc_type,
            body={"size":10000000,"query":self.booling})
        return conclution['hits']['hits']

In [2]:
ES = esf(["ptt","facebook"])    

In [3]:
doc_type=ES.doc_type

In [4]:
QU=esc("ptt",must_cond=["柯","p"],mustnot_cond="連",should_cond="小英",time_sort=True,date1="2015-01-15",date2=None)

{'filtered': {'filter': {'bool': {'must_not': [{'match_phrase': {'_all': '連'}}], 'must': [{'match_phrase': {'_all': '柯'}}, {'match_phrase': {'_all': 'p'}}, {'range': {'time': {'lte': None, 'gte': '2015-01-15'}}}], 'should': [{'match_phrase': {'_all': '小'}}, {'match_phrase': {'_all': '英'}}]}}}}


In [5]:
QU

In [6]:
QU.count()

84

In [8]:
QU = esc("ptt" ,
      must_cond = ["柯","北門","市"],
      mustnot_cond = "連",
      should_cond = ["歷史","橋"],
      time_sort = True,
      date1 ="2015-01-15" , date2 = "2016-06-15" ) 

{'filtered': {'filter': {'bool': {'must_not': [{'match_phrase': {'_all': '連'}}], 'must': [{'match_phrase': {'_all': '柯'}}, {'match_phrase': {'_all': '北門'}}, {'match_phrase': {'_all': '市'}}, {'range': {'time': {'lte': '2016-06-15', 'gte': '2015-01-15'}}}], 'should': [{'match_phrase': {'_all': '歷史'}}, {'match_phrase': {'_all': '橋'}}]}}}}


In [9]:
QU.result(1)

[{'_id': 'AVMcjoTpuidTCCE7SoxE',
  '_index': 'platform',
  '_score': None,
  '_source': {'': '71',
   'content': 'https://www.facebook.com/DoctorKoWJ/?fref=nf\n\n忠孝橋引道從今(7)日凌晨起開始拆除，希望在大年初七(14)日前完成長達750公尺的引道\n\n拆除作業。未來配合西區門戶計畫的推動，北門及臺北車站將會有新的風貌呈現在世人面\n\n前。\n\n為減輕拆橋施工對交通的衝擊，市府把握春節年假的時候，以「全區同步拆除」的方式完\n\n成拆橋作業，對市民影響降至最低。感謝市府相關同仁犧牲假期投入拆除工作，臺北市的\n\n改造就從這一個關鍵的工程開始，我們一起為臺北市的城市發展寫下新頁。\n\n值此同時，臺南還有許多同胞因為震災而受困於災區中，北市消防局搜救隊昨日即刻南下\n\n支援，迄今仍在努力中。各縣市紛紛投入搶救，臺灣是生命共同體，北市府已盤點相關人\n\n力資源並與台南市政府保持聯繫，全力支援。\n\n對於天災，我們永遠只能「勿恃敵之不來，恃吾有以待之」。大家加油，天佑臺灣！\n\n施工期間進入臺北市區建議多加利用捷運路網，工程及交通改道相關資訊可透過以下網站\n查閱：\n\n臺北好行 http://its.taipei.gov.tw/1.html\n臺北市政府 http://www.taipei.gov.tw/\n臺北市政府交通局 http://www.dot.gov.taipei/\n臺北市政府工務局新建工程處 http://nco.gov.taipei/\n\n詳細公車改道動線可上以下網站查閱：\n\n臺北市公共運輸處 http://www.pto.gov.taipei/\n大臺北公車動態資訊系統 http://ebus.gov.taipei/\n臺北市政府工務局新建工程處 http://nco.gov.taipei/\n\n理性勿戰 不要想說地震不會震到你 要想說如果來了你有什麼準備 不要再噴口水了\n\n--\n史http://imgur.com/U6SagY4 http://imgur.com/sGj9BrV http://imgur.com/tsJDggK\n上http://imgur

'/Users/huangming-yi/Google 雲端硬碟/Team_FloodFire/elasticsearch'

In [10]:
%save ./esclass/esuse.py 1 

File `./esclass/esuse.py` exists. Overwrite (y/[N])?  y
The following commands were written to file `./esclass/esuse.py`:
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

class esf:
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    es_index=IndicesClient(es)
    index = "platform"
    def __init__(self,doc_type):
        self.index = self.index
        self.doc_type = doc_type
    def must(self,must_cond,y):
        query = []
        for cond in must_cond:
            query.append({"match_phrase":{"_all":cond}},)
        query.append(y)
        return query

    def should(self,should_cond):
        query = []
        for cond in should_cond:
            query.append({"match_phrase":{"_all":cond}},)
        return query

    def must_not(self,mustnot_cond):
        query = []
        for cond in mustnot_cond:
            query.append({"match_phrase":{"_all":cond}},)
        return query
    
class esc(esf):
    es = esf.es
    es_